# **XGBoost Implementation with Fashion MNIST**

In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
import xgboost as xgb

### *For reproducibility

In [2]:

np.random.seed(42)


## 1. Data Loading and Preprocessing

In [3]:
# loading dataset
train_data = pd.read_csv('../data/fashion-mnist_train.csv')
test_data = pd.read_csv('../data/fashion-mnist_test.csv')

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']
X_test = test_data.drop('label', axis=1)
y_test = test_data['label']

# Normalize pixel values to [0, 1] ~ good esp for gradient based methods
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


## 2. Model Training with XGBoost

### Define hyperparameters

In [4]:

params = {
    'objective': 'multi:softmax',
    'num_class': 10,
    'max_depth': 6,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'merror'
}

### Train the model

In [5]:
num_round = 100
model = xgb.train(params, dtrain, num_round)

## 3. Model Evaluation

In [6]:
train_preds = model.predict(dtrain)
train_accuracy = accuracy_score(y_train, train_preds)
print(f"Train Accuracy: {train_accuracy:.4f}")

test_preds = model.predict(dtest)
test_accuracy = accuracy_score(y_test, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

Train Accuracy: 0.9995
Test Accuracy: 0.8982


In [7]:
preds = model.predict(dtest)
print("Classification Report:")
print(classification_report(y_test, preds))

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.87      0.85      1000
           1       0.99      0.97      0.98      1000
           2       0.80      0.84      0.82      1000
           3       0.90      0.91      0.91      1000
           4       0.82      0.84      0.83      1000
           5       0.99      0.97      0.98      1000
           6       0.74      0.68      0.71      1000
           7       0.95      0.97      0.96      1000
           8       0.98      0.97      0.97      1000
           9       0.96      0.97      0.97      1000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



In [8]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, preds))

Confusion Matrix:
[[866   0  17  17   3   1  90   0   6   0]
 [  3 973   1  15   4   0   3   0   1   0]
 [ 16   1 841   7  76   0  58   0   1   0]
 [ 18   3  16 912  24   0  25   0   2   0]
 [  0   0  79  30 836   0  53   0   2   0]
 [  0   0   1   0   0 971   0  17   0  11]
 [125   1  90  24  73   1 675   0  11   0]
 [  0   0   0   0   0   6   0 968   0  26]
 [  1   1   5   3   3   2  10   4 971   0]
 [  0   0   0   0   0   2   0  28   1 969]]


## 4. Data Preprocessing: PCA

In [13]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import xgboost as xgb

train_data = pd.read_csv('../data/fashion-mnist_train.csv')
test_data = pd.read_csv('../data/fashion-mnist_test.csv')

X_train = train_data.drop('label', axis=1)
y_train = train_data['label']
X_test = test_data.drop('label', axis=1)
y_test = test_data['label']

# Combine train and test for unified PCA transform
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# Split train/test again (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA to reduce dimensionality (e.g., keep 95% variance)
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Original dim: {X_train.shape[1]}, Reduced dim: {X_train_pca.shape[1]}")

# Train XGBoost classifier
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
model.fit(X_train_pca, y_train)

y_pred = model.predict(X_test_pca)


Original dim: 784, Reduced dim: 256


/opt/anaconda3/envs/xgb_env/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [00:25:38] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### Evaluation

In [ ]:
y_train_pred = model.predict(X_train_pca)
train_acc = accuracy_score(y_train, y_train_pred)
print(f"Train Accuracy: {train_acc:.4f}")

test_acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_acc:.4f}")

Train Accuracy: 0.9999
Test Accuracy: 0.8786


In [15]:
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.85      0.84      1400
           1       0.99      0.97      0.98      1400
           2       0.79      0.80      0.80      1400
           3       0.88      0.91      0.90      1400
           4       0.79      0.83      0.81      1400
           5       0.95      0.94      0.95      1400
           6       0.70      0.64      0.67      1400
           7       0.94      0.94      0.94      1400
           8       0.97      0.96      0.96      1400
           9       0.94      0.96      0.95      1400

    accuracy                           0.88     14000
   macro avg       0.88      0.88      0.88     14000
weighted avg       0.88      0.88      0.88     14000



In [16]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Confusion Matrix:
 [[1193    1   19   34    3    0  139    1    9    1]
 [   2 1352    2   32    4    1    5    0    2    0]
 [  16    2 1116   18  138    1  106    0    3    0]
 [  29    2   13 1271   36    0   44    0    5    0]
 [   6    1  106   44 1158    0   79    0    6    0]
 [   1    0    0    0    0 1314    0   49    5   31]
 [ 191    2  143   36  116    1  899    0   11    1]
 [   0    0    0    0    0   33    0 1316    2   49]
 [   6    1    5    4   11    9   18    3 1341    2]
 [   0    0    0    0    1   20    0   37    1 1341]]


## 5. Data Preprocessing: HOG

In [38]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import hog

df_train = pd.read_csv("../data/fashion-mnist_train.csv")
df_test = pd.read_csv("../data/fashion-mnist_test.csv")
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Train: 60000 rows of 1 label + 784 pixels to be converted into a 28x28 image
Test: 10000 rows of 1 label + 784 pixels to be converted into a 28x28 image

In [22]:
fashion_mnist_labels = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

In [23]:
def extract_hog_features(images):
    hog_features = []
    for img in images:
        fd = hog(img, pixels_per_cell=(4, 4), cells_per_block=(2, 2), orientations=9, block_norm='L2-Hys')
        hog_features.append(fd)
    return np.array(hog_features)

In [30]:
# Extract all pixel values (drop label column)
X_train_raw = df_train.iloc[:, 1:].values  # feature shape (60000rows, 784 col)
y_train = df_train.iloc[:, 0].values       # label shape (60000rows,1 col)


X_train_images = X_train_raw.reshape(-1, 28, 28) # Reshape all rows to (28, 28)


X_train_images = X_train_images / 255.0 # Normalize

# Extract HOG features from first 1000 images (for speed)
X_hog = extract_hog_features(X_train_images[:1000])

# X_hog = extract_hog_features(X_train_images)  for full dataset
y_hog = y_train[:1000]

xgb_hog = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_hog.fit(X_hog, y_hog)

X_train_hog, X_test_hog, y_train_hog, y_test_hog = train_test_split(
    X_hog, y_hog, test_size=0.2, random_state=42
)

xgb_hog.fit(X_train_hog, y_train_hog)

/opt/anaconda3/envs/xgb_env/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [00:35:19] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/envs/xgb_env/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [00:35:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

### Evaluation

In [29]:
y_train_pred = xgb_hog.predict(X_train_hog)
train_acc = accuracy_score(y_train_hog, y_train_pred)
print(f"Train Accuracy: {train_acc:.4f}")

# Predict on test set
y_test_pred = xgb_hog.predict(X_test_hog)
test_acc = accuracy_score(y_test_hog, y_test_pred)
print(f"Test Accuracy: {test_acc:.4f}")

Train Accuracy: 1.0000
Test Accuracy: 0.8250


In [28]:
y_pred_hog = xgb_hog.predict(X_test_hog)
print("XGBoost Performance on HOG Features:")
print(classification_report(y_test_hog, y_pred_hog))

XGBoost Performance on HOG Features:
              precision    recall  f1-score   support

           0       0.90      0.78      0.84        23
           1       0.95      0.95      0.95        20
           2       0.71      0.79      0.75        19
           3       0.85      0.89      0.87        19
           4       0.75      0.82      0.78        22
           5       0.92      0.57      0.71        21
           6       0.72      0.65      0.68        20
           7       0.81      0.89      0.85        28
           8       1.00      1.00      1.00        16
           9       0.71      1.00      0.83        12

    accuracy                           0.82       200
   macro avg       0.83      0.83      0.83       200
weighted avg       0.83      0.82      0.82       200



In [37]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test_hog, y_pred_hog))


Confusion Matrix:
 [[18  0  1  0  0  0  4  0  0  0]
 [ 0 19  0  1  0  0  0  0  0  0]
 [ 0  0 15  0  4  0  0  0  0  0]
 [ 1  1  0 17  0  0  0  0  0  0]
 [ 0  0  3  0 18  0  1  0  0  0]
 [ 0  0  0  0  0 12  0  6  0  3]
 [ 1  0  2  2  2  0 13  0  0  0]
 [ 0  0  0  0  0  1  0 25  0  2]
 [ 0  0  0  0  0  0  0  0 16  0]
 [ 0  0  0  0  0  0  0  0  0 12]]


## 6. Data Preprocessing: SIFT

In [31]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    all_features = []

    for img in images:
        kp, des = sift.detectAndCompute(img, None)
        if des is not None:
            # Take mean of all descriptors in this image
            feature = np.mean(des, axis=0)
        else:
            # No keypoints found; use zeros
            feature = np.zeros(128)
        all_features.append(feature)

    return np.array(all_features)

In [32]:
X_raw = df_train.iloc[:, 1:].values.reshape(-1, 28, 28).astype(np.uint8)
y = df_train.iloc[:, 0].values

X_sift = extract_sift_features(X_raw)

print("Shape of SIFT feature matrix:", X_sift.shape) 

Shape of SIFT feature matrix: (60000, 128)


In [35]:
X_train_sift, X_test_sift, y_train_sift, y_test_sift = train_test_split(
    X_sift, y, test_size=0.2, random_state=42
)

# Step 2: Train XGBoost on SIFT features
xgb_sift = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_sift.fit(X_train_sift, y_train_sift)

# Step 3: Predict and evaluate
y_pred_sift = xgb_sift.predict(X_test_sift)

/opt/anaconda3/envs/xgb_env/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [00:39:25] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### Evaluation

In [36]:
y_train_pred_sift = xgb_sift.predict(X_train_sift)
train_acc_sift = accuracy_score(y_train_sift, y_train_pred_sift)
print(f"Train Accuracy (SIFT): {train_acc_sift:.4f}")

test_acc_sift = accuracy_score(y_test_sift, y_pred_sift)
print(f"Test Accuracy (SIFT): {test_acc_sift:.4f}")

Train Accuracy (SIFT): 0.9381
Test Accuracy (SIFT): 0.6548


In [34]:
print("XGBoost Performance on SIFT Features:")
print(f"Accuracy: {accuracy_score(y_test_sift, y_pred_sift):.4f}")
print(classification_report(y_test_sift, y_pred_sift))

XGBoost Performance on SIFT Features:
Accuracy: 0.6548
              precision    recall  f1-score   support

           0       0.66      0.59      0.62      1232
           1       0.64      0.90      0.75      1174
           2       0.55      0.52      0.54      1200
           3       0.60      0.57      0.58      1242
           4       0.53      0.52      0.53      1185
           5       0.77      0.82      0.79      1141
           6       0.41      0.34      0.37      1243
           7       0.82      0.76      0.79      1224
           8       0.69      0.71      0.70      1149
           9       0.86      0.85      0.86      1210

    accuracy                           0.65     12000
   macro avg       0.65      0.66      0.65     12000
weighted avg       0.65      0.65      0.65     12000



In [38]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test_sift, y_pred_sift))


Confusion Matrix:
 [[ 724   79   50   91   28   36  121   14   69   20]
 [   7 1052   10   57    7    9   12    6   14    0]
 [  28   56  627   33  204   36  151    7   55    3]
 [  80  157   45  702   51   40   73   25   55   14]
 [  23   47  192   68  621   22  156    6   50    0]
 [  19   47    6   10    5  938   11   77   13   15]
 [ 172   83  152   92  207   27  421    7   66   16]
 [  11   54    6   28   10   67   22  931   11   84]
 [  21   53   49   72   36   30   57   10  811   10]
 [  18   16    3   22    6   21   10   56   27 1031]]
